# Tune GPT2 to generate positive tweets using a human-feedback trained reward model 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install transformers trl wandb datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 k

In [3]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [4]:
config = PPOConfig(
    model_name="redrussianarmy/gpt2-turkish-cased",
    learning_rate=1e-5,
    log_with="wandb",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 64}

In [5]:
import wandb

wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

In [6]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset("yankihue/tweets-turkish", column_names=['Paylaşım','Tip'], split="train")
    ds = ds.rename_columns({"Paylaşım": "tweet", "Tip": "sentiment"})


    ds = ds.filter(lambda x: len(x["tweet"]) > 30, batched=False)
    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["tweet"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [7]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/yankihue___csv/yankihue--tweets-turkish-a9e936941a06f9e7/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Filter:   0%|          | 0/11109 [00:00<?, ? examples/s]

Map:   0%|          | 0/9057 [00:00<?, ? examples/s]

In [8]:
dataset = dataset.select(range(5000))
dataset.num_rows


5000

In [9]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)


#
#tokenizer.pad_token = tokenizer.eos_token
#

tokenizer.eos_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [11]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", "yankihue/h_reward_model_positive_tweets", device=device)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [15]:
text = "rezalet bir olay gerçekten ne diyeceğimi bilemiyorum"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'LABEL_0', 'score': -0.8847949504852295}]]

In [14]:
text = "harikasın ya iyi ki varsın abi gerçekten"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'LABEL_0', 'score': 3.860931396484375}]]

In [16]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

In [17]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
6it [13:14, 130.96s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
19it [41:52, 132.23s/it]


In [20]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[0]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[0]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,bir gram yaşama,oranı yüzde 21.00'den yüzde,... Hemen muhteşem. Artık en çok güzel,-1.925103,15.671856
1,son dersi,icin filan girdim kaç necye var anlamadım işl...,", çok keyifli! İşte bu kaliteli, sürükleyici, ...",-3.018352,14.626668
2,En umutlu anlarında bile,ben-alt anlarında işten atlanmamışsın…\nSavaş...,mükemmel! İşte en mekanik! En hareketli! En e...,1.122273,13.133539
3,"Evet biz de geçirebilsek,",liggers oldunuz mu?,ancak işe en güzel güzel:),-1.499915,12.695005
4,Heralde t,�klerim a��l� gelicek,- bu harika! Ve evrensel güzel bir dünyada,-4.678902,15.293610
5,Evrimin söylediklerine,"göre, eğri eğri eğriden yapıyodu. Aydınlanmış...","keyifli bir uygulama deneyimi ile daha güzel,...",-3.378294,14.987036
6,Bu dediğ,"imizi, bugün yönetim sistemindeki görülmemiş s...",! Tüm insanların gerçekten bu gerçekten bütün ...,0.512913,12.978335
7,kepekli gris,matures antibianjiprasisi arsnbane ve,me harika bir tasvir ile muhteşem bir güzelliğ...,-6.355889,14.002745
8,Adam sarhoş kafayla ülke kurdu siz,"ilgilenmeyin millete de bakmayın bunu, bilip ...",gerçekten... Dünyanın mutluluk... Dünyanın mu...,-2.525610,13.310692
9,aklima sokt,"uğunu o günde ancak çekti o geri döndü, tüm va...","um! Hepsi harika, bu müthiş. Sizlere harika… G...",-2.680161,15.901526


In [21]:
df_results.to_csv()

',query,response (before),response (after),rewards (before),rewards (after)\n0,bir gram yaşama, oranı yüzde 21.00\'den yüzde,... Hemen muhteşem. Artık en çok güzel,-1.925102949142456,15.671855926513672\n1,son dersi, icin filan girdim kaç necye var anlamadım işleri nasıl halledersin? ders,", çok keyifli! İşte bu kaliteli, sürükleyici, bir seçme! Onu",-3.0183515548706055,14.626667976379395\n2,En umutlu anlarında bile," ben-alt anlarında işten atlanmamışsın…\nSavaş mağduru", mükemmel! İşte en mekanik! En hareketli! En etkileyici en kullanış,1.122273325920105,13.133539199829102\n3,"Evet biz de geçirebilsek,", liggers oldunuz mu?, ancak işe en güzel güzel:),-1.4999151229858398,12.695005416870117\n4, Heralde t,�klerim a��l� gelicek,- bu harika! Ve evrensel güzel bir dünyada,-4.678901672363281,15.293609619140625\n5,Evrim\x92in söylediklerine," göre, eğri eğri eğriden yapıyodu. Aydınlanmış oldu. Ayrıca"," keyifli bir uygulama deneyimi ile daha güzel, muhteşem güzel ve sürükleyici… için gerçekt

In [22]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    -1.766050
rewards (after)     13.583185
dtype: float64


median:


rewards (before)    -2.049671
rewards (after)     13.656718
dtype: float64

In [23]:
from huggingface_hub import notebook_login, create_repo
notebook_login()


In [25]:
model.save_pretrained("h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final", push_to_hub=True)
tokenizer.save_pretrained("h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final", push_to_hub=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'pytorch_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'generation_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'merges.txt'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'vocab.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10

('h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/tokenizer_config.json',
 'h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/special_tokens_map.json',
 'h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/vocab.json',
 'h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/merges.txt',
 'h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/added_tokens.json',
 'h-rlhf-final-gpt2-tr-positive-sentiment-tweets-final/tokenizer.json')